## <center>BMEIS Workshops</center>

# <center>Testing</center>

### <center>Eric Kerfoot, Ben Murray, Marc Modat</center>
### <center>School of Biomedical Engineering & Imaging Sciences</center>
### <center>King's College London</center>

# Testing


## Aim:
* Learn the basic concepts and processes behind testing, unit testing, and test driven development

## What We'll Cover
* Basic Testing Idea
  * Why we test, need for demonstrable correctness, increase confidence in code, minimize costs associated with failure
  * Designing for testability
  * Error, fault, failure
  * Test driven development, test-orientation
  
* Unit Tests
  * What is meant to be tested
  * Cases, suite, framework
  * Equivalent tests   

### Why test?

* Formal way of quantifying correctness
* Defines criteria for correctness which software should meet
* Need to show correctness, increase confidence in code
* Decreases cost of failure
* **"Testing shows the presence, not the absence of bugs"** <br/>- Edsger Dijkstra

### Basic example: Testing Output

In [1]:
def sqrt(a):
    return a ** 0.5

if sqrt(4) != 2: # simple test
    raise ValueError('Bad value!')

#### Software specification

* Defines the requirements for the software being developed
* Contains functional and other criteria the software must meet to be considered successful
* Correctness is always one of these, hard to assert any others are met if not correct
* Many requirements define what tests should be run, these are used to demonstrate the specification is met

* **Error**: When the behaviour of the program differs from that stated in the specification
* **Fault**: What component in the software caused the error to occur (ie. defective code or "bug")
* **Failure**: When the software cannot perform its specified function
----

### Testing Objective:  <font color="red">Identify faults by detecting when errors occur to minimize the frequency and cost of failure</font>

### Design for testability

* Develop code and systems to be composed of testable units
* Encourages good practice by enforcing the notion of modularity and composition of components
* Discourages defining software which is hard to show is correct or meets specification
* Testable code tends to be maintainable and well designed

### Asserting correctness

* Python includes an `assert` statement to check a condition is true at runtime
* Used to ensure internal algorithm state is correct at certain stages
* Not used to check external input or other conditions beyond the definition of the algorithm
* Syntax: `assert C [,message]`: throws an exception is `C` evaluates to False with `message` as the exception message

In [2]:
def binary_search(items,item):
    first, last = 0, len(items) - 1
    mid = last // 2
    
    while first <= last and items[mid] != item :
        mid = (first + last) // 2
        
        assert 0 <= mid < len(items), '`mid` out of bounds!'
        
        if item < items[mid]:
            last = mid - 1
        else:
            first = mid + 1
                
    return mid

binary_search([1,3,4,6,7,12,18,25],12)

5

* We use assertions to show algorithms are behaving as expected
* Can be turned off at runtime for efficiency
* Doesn't test the whole algorithm/routine/module
* Unit tests are the common method for doing this instead

## Unit Testing

* A unit test is a simple routine/program to test a property of a small unit of code
* **Test case**: a set of inputs and expected outputs to test one of these small units
* **Test Suite**: a set of test cases for a routine/module/system
* **Test Framework/Harness**: supporting software environment around test suites which provide services to facilitate the testing process

### Test Case
* Components:
  * Tested code unit
  * Test input data
  * Expected output data
  * Comparison criteria to assess actual output in relation to expected output

* Eg. Simple function calculating the square root of a given float:

In [3]:
def sqrt(a):
    return a ** 0.5

* Need values for `a` which demonstrate `sqrt` returns the correct result
* Need other values to demonstrate it will handle incorrect values appropriately

* We'll use `unittest` standard library to define our test cases:

In [4]:
import unittest

class SqrtTests(unittest.TestCase): # harness class
    def test_correct1(self): # actual test case
        b = sqrt(4) # input data is 4
        self.assertEqual(b, 2) # expected output is 2
        
unittest.main(argv=['ignored'], exit=False) # needed for Jupyter

.
----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


* What does this show?
* For one correct input `sqrt` behaved correctly
* What about any other correct inputs? What correct inputs wouldn't be equivalent to `4`?
* Consider the edge cases for this function, eg. `0`

In [5]:
import unittest

class SqrtTests(unittest.TestCase): # harness class
    def test_correct1(self): # actual test case
        b = sqrt(0) # input data is 4
        self.assertEqual(b, 0) # expected output is 2
        
unittest.main(argv=['ignored'], exit=False) 

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


* What about negative inputs?

In [6]:
class SqrtTests(unittest.TestCase): 
    def test_negative1(self): 
        b = sqrt(-4) # input data is -4
        # expected output is -2 (bear with me on this one)
        self.assertEqual(b, -2) 
        
unittest.main(argv=['ignored'], exit=False)

F
FAIL: testNegative1 (__main__.SqrtTests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-6-213e4ac906ef>", line 5, in testNegative1
    self.assertEqual(b, -2)
AssertionError: (1.2246467991473532e-16+2j) != -2

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (failures=1)


* Returning a complex number is numerically correct, but is this what we wanted?
* If `sqrt` was supposed to operate only on real numbers this constitutes an error and a fault exists
* The fault could be the lack of input sanitation:

In [7]:
def sqrt(a):
    if a<0: # remember, don't use assert to do this
        raise ValueError('Negative value for `a`')
        
    return a ** 0.5

* Now our test is to show the exception is raised under the correct conditions:

In [8]:
class SqrtTests(unittest.TestCase): 
    def test_negative1(self): 
        with self.assertRaises(ValueError): # exception caught here
            b = sqrt(-4) 
        
unittest.main(argv=['ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


* What about complex input? Positive or negative infinity? NaN?
* Answer: write more test cases!


* Test harnesses are used to setup and tear down the testing environment test cases may require:

In [9]:
class SqrtTests(unittest.TestCase):
    def setUp(self):
        self.correctValues=[0,1,4,3,100,3.14159]
        self.correctOutputs=[0,1,2,1.732,10,1.772]
        
    def tearDown(self):
        pass # close file handles, delete files, etc.
    
    def test_negative1(self): 
        for x,y in zip(self.correctValues,self.correctOutputs):
            self.assertAlmostEqual(sqrt(x),y,3) # equal to 3 decimals
            
unittest.main(argv=['ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Test Drive Development
* Always be writing test cases as you develop, use them instead of little tester scripts
* Design your software with a view to testability, thinking about how to write tests for each component as you go
* Developing test cases is a creative process, but consider what tests are equivalent to others and discard them, and think about testing the edge cases
* Test to fail, don't write easy tests just to pass

## PyCharm
* We'll start using the PyCharm IDE now
* When you start the program open the directory containing the exercise, this will open that directory as a project
* The exercises are given as .py files rather notebooks now.
* Running unit tests and code coverage is done through its interface
* `unittest` can also be run (from the root of your source file hierarchy) with `python -m unittest`

# That's it!

## Questions?

On to practicals...